In [18]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.append(str(PROJECT_ROOT))

from src.config import PoseConfig, CurlConfig, SquatConfig, ShoulderPressConfig
from src.pose_tracker import PoseTracker
from src.counters import BicepCurlCounter, SquatCounter, ShoulderPressCounter
import cv2


In [20]:
# =========================
# POSE TRAINER (curl/squat/press)
# Start gate runs ONLY ONCE
# Shows Form Confidence + Alignment
# Final report includes suggestions
# FIXED: always uses waitKey to prevent "Not Responding"
# =========================

pose_cfg = PoseConfig()
curl_cfg = CurlConfig()
squat_cfg = SquatConfig()
press_cfg = ShoulderPressConfig()

tracker = PoseTracker(
    min_det_conf=pose_cfg.min_detection_confidence,
    min_track_conf=pose_cfg.min_tracking_confidence
)

mp_pose = __import__("mediapipe").solutions.pose
from src.utils import calculate_angle

# ---------- USER INPUT ----------
choice = input("Enter exercise (curl/squat/press): ").strip().lower()
TARGET_REPS = int(input("Enter target reps: ").strip())

if choice == "curl":
    EXERCISE_NAME = "Bicep Curl (Left Arm)"
    counter = BicepCurlCounter(
        up_angle=curl_cfg.up_angle,
        down_angle=curl_cfg.down_angle,
        visibility_threshold=curl_cfg.visibility_threshold
    )
elif choice == "squat":
    EXERCISE_NAME = "Squat"
    counter = SquatCounter(
        up_angle=squat_cfg.up_angle,
        down_angle=squat_cfg.down_angle,
        visibility_threshold=squat_cfg.visibility_threshold
    )
elif choice == "press":
    EXERCISE_NAME = "Shoulder Press"
    counter = ShoulderPressCounter(
        up_angle=press_cfg.up_angle,
        down_angle=press_cfg.down_angle,
        visibility_threshold=press_cfg.visibility_threshold
    )
else:
    raise ValueError("Invalid input. Type only: curl, squat, press")

# ---------- START CONDITION ----------
START_VIS_THRESH = 0.55
READY_FRAMES_REQUIRED = 6
started = False
ready_frames = 0

# ---------- TRACKING ----------
form_scores = []
align_scores = []
last_angle = None

def clamp01(x):
    return max(0.0, min(1.0, x))

def score_from_error(err, tol):
    return clamp01(1.0 - (err / tol))

def alignment_score(choice, lm):
    def pt(e):
        p = lm[e.value]
        return (p.x, p.y)

    LS = pt(mp_pose.PoseLandmark.LEFT_SHOULDER)
    RS = pt(mp_pose.PoseLandmark.RIGHT_SHOULDER)
    LH = pt(mp_pose.PoseLandmark.LEFT_HIP)
    RH = pt(mp_pose.PoseLandmark.RIGHT_HIP)

    mid_sh = ((LS[0] + RS[0]) / 2, (LS[1] + RS[1]) / 2)
    mid_hip = ((LH[0] + RH[0]) / 2, (LH[1] + RH[1]) / 2)

    down = (mid_hip[0], mid_hip[1] + 0.3)
    torso_angle = calculate_angle(mid_sh, mid_hip, down)
    torso_err = abs(180 - torso_angle)
    torso_score = score_from_error(torso_err, tol=25)

    if choice == "curl":
        LE = pt(mp_pose.PoseLandmark.LEFT_ELBOW)
        LW = pt(mp_pose.PoseLandmark.LEFT_WRIST)

        elbow_hip_dx = abs(LE[0] - LH[0])
        elbow_close_score = score_from_error(elbow_hip_dx, tol=0.12)

        wrist_elbow_dx = abs(LW[0] - LE[0])
        wrist_stack_score = score_from_error(wrist_elbow_dx, tol=0.10)

        s = 0.45 * torso_score + 0.35 * elbow_close_score + 0.20 * wrist_stack_score
        return 100 * s

    if choice == "squat":
        LK = pt(mp_pose.PoseLandmark.LEFT_KNEE)
        RK = pt(mp_pose.PoseLandmark.RIGHT_KNEE)
        LA = pt(mp_pose.PoseLandmark.LEFT_ANKLE)
        RA = pt(mp_pose.PoseLandmark.RIGHT_ANKLE)

        left_track = score_from_error(abs(LK[0] - LA[0]), tol=0.10)
        right_track = score_from_error(abs(RK[0] - RA[0]), tol=0.10)
        knee_track = (left_track + right_track) / 2

        mid_ank = ((LA[0] + RA[0]) / 2, (LA[1] + RA[1]) / 2)
        hip_center_err = abs(mid_hip[0] - mid_ank[0])
        hip_center = score_from_error(hip_center_err, tol=0.12)

        s = 0.45 * knee_track + 0.35 * torso_score + 0.20 * hip_center
        return 100 * s

    if choice == "press":
        LE = pt(mp_pose.PoseLandmark.LEFT_ELBOW)
        LW = pt(mp_pose.PoseLandmark.LEFT_WRIST)
        RE = pt(mp_pose.PoseLandmark.RIGHT_ELBOW)
        RW = pt(mp_pose.PoseLandmark.RIGHT_WRIST)

        left_stack = score_from_error(abs(LW[0] - LE[0]), tol=0.10)
        right_stack = score_from_error(abs(RW[0] - RE[0]), tol=0.10)
        stack_score = (left_stack + right_stack) / 2

        sh_height_err = abs(LS[1] - RS[1])
        symmetry = score_from_error(sh_height_err, tol=0.06)

        s = 0.45 * stack_score + 0.35 * torso_score + 0.20 * symmetry
        return 100 * s

    return 0.0

cap = cv2.VideoCapture(pose_cfg.camera_index)
if not cap.isOpened():
    raise RuntimeError("Camera could not be opened. Try changing PoseConfig.camera_index to 0 or 1.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results, annotated = tracker.process_bgr(frame)

    # Always draw header
    cv2.putText(annotated, f"Exercise: {EXERCISE_NAME}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.putText(annotated, f"Target: {TARGET_REPS}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    # ---------- START GATE (ONLY UNTIL STARTED) ----------
    if not started:
        if not results.pose_landmarks:
            ready_frames = 0
            cv2.putText(annotated, "POSE NOT DETECTED - step back, full body in frame", (10, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 2)

            cv2.imshow("Pose Trainer", annotated)
            key = cv2.waitKey(10) & 0xFF
            if key == ord('q'):
                break
            continue

        lm = results.pose_landmarks.landmark

        KEY_LMS = [
            mp_pose.PoseLandmark.NOSE,
            mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER,
            mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP,
            mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.RIGHT_KNEE,
            mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.RIGHT_ANKLE,
        ]

        key_visible = all(
            getattr(lm[e.value], "visibility", 0.0) >= START_VIS_THRESH
            for e in KEY_LMS
        )

        ready_frames = (ready_frames + 1) if key_visible else 0

        cv2.putText(annotated, "Stand ready in full frame", (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        cv2.putText(annotated, f"Getting ready: {ready_frames}/{READY_FRAMES_REQUIRED}",
                    (10, 155), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)

        if ready_frames >= READY_FRAMES_REQUIRED:
            started = True
            cv2.putText(annotated, "START ✅", (10, 195),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 3)

            cv2.imshow("Pose Trainer", annotated)
            cv2.waitKey(600)  # short pause so user sees START
        else:
            cv2.imshow("Pose Trainer", annotated)
            key = cv2.waitKey(10) & 0xFF
            if key == ord('q'):
                break
            continue

    # ---------- AFTER START ----------
    # If pose disappears, show message but keep window responsive
    if not results.pose_landmarks:
        cv2.putText(annotated, "Pose lost - stay in frame", (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
        cv2.imshow("Pose Trainer", annotated)
        key = cv2.waitKey(10) & 0xFF
        if key == ord('q'):
            break
        continue

    lm = results.pose_landmarks.landmark

    # Alignment (live)
    align_pct = alignment_score(choice, lm)
    align_scores.append(align_pct)
    avg_align_live = sum(align_scores) / max(1, len(align_scores))

    # Rep counting
    if choice in ("curl", "press"):
        shoulder = (lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                    lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
        elbow = (lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                 lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].y)
        wrist = (lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                 lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y)

        vis = lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
        info = counter.update(shoulder, elbow, wrist, vis)
    else:
        hip = (lm[mp_pose.PoseLandmark.LEFT_HIP.value].x,
               lm[mp_pose.PoseLandmark.LEFT_HIP.value].y)
        knee = (lm[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                lm[mp_pose.PoseLandmark.LEFT_KNEE.value].y)
        ankle = (lm[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                 lm[mp_pose.PoseLandmark.LEFT_ANKLE.value].y)

        vis = lm[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
        info = counter.update(hip, knee, ankle, vis)

    # Form confidence (performance)
    angle = info["angle"]
    if choice == "curl":
        good_range = 20 <= angle <= 160
    elif choice == "squat":
        good_range = 80 <= angle <= 180
    else:
        good_range = 70 <= angle <= 180

    score = 70 if good_range else 0
    if last_angle is not None:
        angle_change = abs(angle - last_angle)
        score += 20 if angle_change < 15 else 5
    last_angle = angle
    if info["reliable"]:
        score += 10

    form_scores.append(min(100, score))
    avg_form_live = sum(form_scores) / max(1, len(form_scores))

    # Display
    cv2.putText(annotated, f"Angle: {angle:.1f}", (10, 110),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.putText(annotated, f"Reps: {info['counter']} / {TARGET_REPS}", (10, 150),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.putText(annotated, f"Form Confidence: {avg_form_live:.1f}%", (10, 190),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.putText(annotated, f"Alignment: {avg_align_live:.1f}%", (10, 230),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)

    cv2.imshow("Pose Trainer", annotated)
    key = cv2.waitKey(10) & 0xFF
    if key == ord('q'):
        break

    if info["counter"] >= TARGET_REPS:
        break

cap.release()
cv2.destroyAllWindows()

# ================== FINAL REPORT WITH SUGGESTIONS ==================
completed_reps = counter.state.counter
avg_form = sum(form_scores) / max(1, len(form_scores))
avg_align = sum(align_scores) / max(1, len(align_scores))

print("\n===== PERFORMANCE REPORT =====")
print(f"Exercise        : {EXERCISE_NAME}")
print(f"Target Reps     : {TARGET_REPS}")
print(f"Completed Reps  : {completed_reps}")
print(f"Form Confidence : {avg_form:.1f}%")
print(f"Alignment       : {avg_align:.1f}%")

print("\n--- SUGGESTIONS ---")
if choice == "curl":
    print("• Keep your elbow close to your body (don’t let it drift forward).")
    print("• Avoid swinging your torso — stay upright.")
    print("• Control the lowering phase (don’t drop quickly).")
    print("• Keep wrist neutral (in line with forearm).")
elif choice == "squat":
    print("• Keep chest up and spine neutral (avoid rounding).")
    print("• Knees track outward (avoid caving inward).")
    print("• Keep heels down and weight mid-foot.")
    print("• Go as deep as comfortable with control.")
elif choice == "press":
    print("• Keep core tight; don’t lean back.")
    print("• Wrists stacked over elbows at the bottom.")
    print("• Press straight overhead, not forward.")
    print("• Keep shoulders level (avoid tilting).")

print("================================\n")


Enter exercise (curl/squat/press):  squat
Enter target reps:  5



===== PERFORMANCE REPORT =====
Exercise        : Squat
Target Reps     : 5
Completed Reps  : 5
Form Confidence : 96.4%
Alignment       : 67.5%

--- SUGGESTIONS ---
• Keep chest up and spine neutral (avoid rounding).
• Knees track outward (avoid caving inward).
• Keep heels down and weight mid-foot.
• Go as deep as comfortable with control.



In [17]:
# =========================
# POSE TRAINER (curl/squat/press)
# Video File Version (no webcam)
# Uses YOUR repo file names:
#   src/config.py
#   src/counters.py
#   src/pose_tracker.py
#   src/utils.py
# =========================

import os
import cv2

from src.config import PoseConfig, CurlConfig, SquatConfig, ShoulderPressConfig
from src.pose_tracker import PoseTracker
from src.counters import BicepCurlCounter, SquatCounter, ShoulderPressCounter
from src.utils import calculate_angle

mp_pose = __import__("mediapipe").solutions.pose

# ---------- FIXED SETTINGS (NO INPUT) ----------
VIDEO_PATH = r"C:\Users\Gurprajas\OneDrive\Desktop\video demo.mp4"

# Reduce display/processing frame size here
RESIZE_W = 540
RESIZE_H = 960

# set this once and run
choice = "squat"     # "curl" / "squat" / "press"
TARGET_REPS = 10

# ---------- CONFIG ----------
pose_cfg = PoseConfig()
curl_cfg = CurlConfig()
squat_cfg = SquatConfig()
press_cfg = ShoulderPressConfig()

tracker = PoseTracker(
    min_det_conf=pose_cfg.min_detection_confidence,
    min_track_conf=pose_cfg.min_tracking_confidence
)

# ---------- COUNTER ----------
if choice == "curl":
    EXERCISE_NAME = "Bicep Curl (Left Arm)"
    counter = BicepCurlCounter(
        up_angle=curl_cfg.up_angle,
        down_angle=curl_cfg.down_angle,
        visibility_threshold=curl_cfg.visibility_threshold
    )
elif choice == "squat":
    EXERCISE_NAME = "Squat"
    counter = SquatCounter(
        up_angle=squat_cfg.up_angle,
        down_angle=squat_cfg.down_angle,
        visibility_threshold=squat_cfg.visibility_threshold
    )
elif choice == "press":
    EXERCISE_NAME = "Shoulder Press"
    counter = ShoulderPressCounter(
        up_angle=press_cfg.up_angle,
        down_angle=press_cfg.down_angle,
        visibility_threshold=press_cfg.visibility_threshold
    )
else:
    raise ValueError("Invalid exercise. Use only: curl, squat, press")

# ---------- START CONDITION ----------
START_VIS_THRESH = 0.55
READY_FRAMES_REQUIRED = 6
started = False
ready_frames = 0

# ---------- TRACKING ----------
form_scores = []
align_scores = []
last_angle = None

def clamp01(x):
    return max(0.0, min(1.0, x))

def score_from_error(err, tol):
    return clamp01(1.0 - (err / tol))

def alignment_score(choice, lm):
    def pt(e):
        p = lm[e.value]
        return (p.x, p.y)

    LS = pt(mp_pose.PoseLandmark.LEFT_SHOULDER)
    RS = pt(mp_pose.PoseLandmark.RIGHT_SHOULDER)
    LH = pt(mp_pose.PoseLandmark.LEFT_HIP)
    RH = pt(mp_pose.PoseLandmark.RIGHT_HIP)

    mid_sh = ((LS[0] + RS[0]) / 2, (LS[1] + RS[1]) / 2)
    mid_hip = ((LH[0] + RH[0]) / 2, (LH[1] + RH[1]) / 2)

    down = (mid_hip[0], mid_hip[1] + 0.3)
    torso_angle = calculate_angle(mid_sh, mid_hip, down)
    torso_err = abs(180 - torso_angle)
    torso_score = score_from_error(torso_err, tol=25)

    if choice == "curl":
        LE = pt(mp_pose.PoseLandmark.LEFT_ELBOW)
        LW = pt(mp_pose.PoseLandmark.LEFT_WRIST)

        elbow_hip_dx = abs(LE[0] - LH[0])
        elbow_close_score = score_from_error(elbow_hip_dx, tol=0.12)

        wrist_elbow_dx = abs(LW[0] - LE[0])
        wrist_stack_score = score_from_error(wrist_elbow_dx, tol=0.10)

        s = 0.45 * torso_score + 0.35 * elbow_close_score + 0.20 * wrist_stack_score
        return 100 * s

    if choice == "squat":
        LK = pt(mp_pose.PoseLandmark.LEFT_KNEE)
        RK = pt(mp_pose.PoseLandmark.RIGHT_KNEE)
        LA = pt(mp_pose.PoseLandmark.LEFT_ANKLE)
        RA = pt(mp_pose.PoseLandmark.RIGHT_ANKLE)

        left_track = score_from_error(abs(LK[0] - LA[0]), tol=0.10)
        right_track = score_from_error(abs(RK[0] - RA[0]), tol=0.10)
        knee_track = (left_track + right_track) / 2

        mid_ank = ((LA[0] + RA[0]) / 2, (LA[1] + RA[1]) / 2)
        hip_center_err = abs(mid_hip[0] - mid_ank[0])
        hip_center = score_from_error(hip_center_err, tol=0.12)

        s = 0.45 * knee_track + 0.35 * torso_score + 0.20 * hip_center
        return 100 * s

    if choice == "press":
        LE = pt(mp_pose.PoseLandmark.LEFT_ELBOW)
        LW = pt(mp_pose.PoseLandmark.LEFT_WRIST)
        RE = pt(mp_pose.PoseLandmark.RIGHT_ELBOW)
        RW = pt(mp_pose.PoseLandmark.RIGHT_WRIST)

        left_stack = score_from_error(abs(LW[0] - LE[0]), tol=0.10)
        right_stack = score_from_error(abs(RW[0] - RE[0]), tol=0.10)
        stack_score = (left_stack + right_stack) / 2

        sh_height_err = abs(LS[1] - RS[1])
        symmetry = score_from_error(sh_height_err, tol=0.06)

        s = 0.45 * stack_score + 0.35 * torso_score + 0.20 * symmetry
        return 100 * s

    return 0.0

# ================= VIDEO CAPTURE =================
if not os.path.exists(VIDEO_PATH):
    raise RuntimeError(f"Video file not found:\n{VIDEO_PATH}")

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError("Could not open video. Check the file path or codec.")

# ================= MAIN LOOP =================
while cap.isOpened():

    ret, frame = cap.read()

    if not ret:
        print("Video finished.")
        break

    # ======= REDUCE VIDEO FRAME SIZE HERE =======
    frame = cv2.resize(frame, (RESIZE_W, RESIZE_H))
    # ===========================================

    results, annotated = tracker.process_bgr(frame)

    cv2.putText(annotated, f"Exercise: {EXERCISE_NAME}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(annotated, f"Target: {TARGET_REPS}", (3, 5),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # ---------- START GATE ----------
    if not started:
        if not results.pose_landmarks:
            ready_frames = 0
            cv2.imshow("Pose Trainer", annotated)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            continue

        lm = results.pose_landmarks.landmark

        KEY_LMS = [
            mp_pose.PoseLandmark.NOSE,
            mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER,
            mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP,
            mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.RIGHT_KNEE,
            mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.RIGHT_ANKLE,
        ]

        key_visible = all(
            getattr(lm[e.value], "visibility", 0.0) >= START_VIS_THRESH
            for e in KEY_LMS
        )

        ready_frames = (ready_frames + 1) if key_visible else 0

        if ready_frames >= READY_FRAMES_REQUIRED:
            started = True
        else:
            cv2.putText(annotated, "Stand fully in frame to start...", (10, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.imshow("Pose Trainer", annotated)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            continue

    # ---------- POSE LOST ----------
    if not results.pose_landmarks:
        cv2.putText(annotated, "Pose lost - get back in frame", (10, 110),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv2.imshow("Pose Trainer", annotated)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        continue

    lm = results.pose_landmarks.landmark

    # ---------- ALIGNMENT ----------
    align_pct = alignment_score(choice, lm)
    align_scores.append(align_pct)
    avg_align_live = sum(align_scores) / len(align_scores)

    # ---------- REP COUNT ----------
    if choice in ("curl", "press"):
        shoulder = (lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                    lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
        elbow = (lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                 lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].y)
        wrist = (lm[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                 lm[mp_pose.PoseLandmark.LEFT_WRIST.value].y)

        vis = lm[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
        info = counter.update(shoulder, elbow, wrist, vis)

    else:  # squat
        hip = (lm[mp_pose.PoseLandmark.LEFT_HIP.value].x,
               lm[mp_pose.PoseLandmark.LEFT_HIP.value].y)
        knee = (lm[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                lm[mp_pose.PoseLandmark.LEFT_KNEE.value].y)
        ankle = (lm[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                 lm[mp_pose.PoseLandmark.LEFT_ANKLE.value].y)

        vis = lm[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
        info = counter.update(hip, knee, ankle, vis)

    # ---------- FORM CONFIDENCE ----------
    angle = info["angle"]

    if choice == "curl":
        good_range = 20 <= angle <= 160
    elif choice == "squat":
        good_range = 80 <= angle <= 180
    else:
        good_range = 70 <= angle <= 180

    score = 70 if good_range else 0
    if last_angle is not None:
        score += 20 if abs(angle - last_angle) < 15 else 5
    last_angle = angle

    if info.get("reliable", False):
        score += 10

    form_scores.append(min(100, score))
    avg_form_live = sum(form_scores) / len(form_scores)

    # ---------- DISPLAY ----------
    cv2.putText(annotated, f"Reps: {info['counter']} / {TARGET_REPS}", (10, 150),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(annotated, f"Form: {avg_form_live:.1f}%", (10, 190),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(annotated, f"Align: {avg_align_live:.1f}%", (10, 230),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(annotated,
                f"Stage: {info.get('stage', None)}  Angle: {angle:.1f}",
                (10, 270),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                (255, 255, 255), 2)

    cv2.imshow("Pose Trainer", annotated)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

    if info["counter"] >= TARGET_REPS:
        break

cap.release()
cv2.destroyAllWindows()

# ================= FINAL REPORT =================
completed_reps = counter.state.counter
avg_form = sum(form_scores) / max(1, len(form_scores))
avg_align = sum(align_scores) / max(1, len(align_scores))

print("\n===== PERFORMANCE REPORT =====")
print(f"Exercise        : {EXERCISE_NAME}")
print(f"Target Reps     : {TARGET_REPS}")
print(f"Completed Reps  : {completed_reps}")
print(f"Form Confidence : {avg_form:.1f}%")
print(f"Alignment       : {avg_align:.1f}%")
print("================================\n")


Video finished.

===== PERFORMANCE REPORT =====
Exercise        : Squat
Target Reps     : 10
Completed Reps  : 4
Form Confidence : 74.1%
Alignment       : 35.9%

